<a href="https://colab.research.google.com/github/felps2003/checkPoint_Cyber/blob/main/CheckPoint_CyberV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instalção do Ngrok para simulação de um servidor**

In [ ]:
!pip install -q pyngrok

     |████████████████████████████████| 745 kB 7.5 MB/s 


**Instalação do framework Streamlit**

In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 10.1 MB 9.9 MB/s 
     |████████████████████████████████| 111 kB 64.2 MB/s 
     |████████████████████████████████| 4.3 MB 56.5 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 181 kB 35.9 MB/s 
     |████████████████████████████████| 164 kB 58.7 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 131 kB 46.0 MB/s 
     |████████████████████████████████| 793 kB 45.0 MB/s 
     |████████████████████████████████| 130 kB 42.0 MB/s 
     |████████████████████████████████| 428 kB 60.5 MB/s 
     |████████████████████████████████| 381 kB 29.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1

In [ ]:
!pip install -q streamlit_ace

     |████████████████████████████████| 3.6 MB 8.9 MB/s 


**Instalação da biblioteca FuzzyWuzzy**

In [ ]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 50 kB 3.8 MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149862 sha256=144c9bd692bf5bbf94e7752102ee92bf8811ef92c1655adf7c5bd17f7375b048
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


In [ ]:
!streamlit run app.py &>/dev/null&

**Importações Necessarias**

In [ ]:
from pyngrok import ngrok

In [ ]:
from fuzzywuzzy import process

Aqui esta o codigo base, nele tive que criar 3 funções:

1.   Função **names** para verificar os nomes, se é suspeito ou não.
2.   Função **verificacaoPassaporte** para verificar os numeros e as letras principais do pais do imigrante.
3.   Função **verificacao** que ira retornar o resultado se a pessoa pode entrar no pais, se ela é suspeita então deve ser presa e caso o pais não conste na base de dados ira aparecer um aviso. 

Quis fazer com que qualquer API que fosse colocada no **res** pudesse ser utilizada, sem precisar mudar as letras principais do codigo, então puxei o que foi escrito no match e caso seja verdadeiro ele ira copiar e então ficara no lugar no X como esse exemplo:

*   x = (api)
*   dict_api = res.json()[x]

Na parte dos nomes escolhi por utilizar o threshold de 80 pois utilizando um menor alguns nomes acabam por passar, e o iraque costuma sofrer com alguns ataques politicos terroristas, então achei melhor manter alto para remediar.


In [ ]:
%%writefile app.py
import streamlit as st
import re
import json
import requests as req
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from pyngrok import ngrok


#res = requests.get('https://henricobela.github.io/fra.json')
nomes = req.get('https://cspinheiro.github.io/interpol.json')
  
def main():

  html_temp = """ 
  <div style ="background-color:red;padding:13px"> 
  <h1 style ="color:black;text-align:center;">Imigração para o Iraque</h1> 
  <h6 style ="color:white;text-align:center;">Por favor digite as três letra de identificação do pais</h6>
  <h6 style ="color:white;text-align:center;">e os numeros do passaporte</h6>
  </div> 
  """      
  st.markdown(html_temp, unsafe_allow_html = True) 
  names = st.text_input('Nome do imigrante')
  numPassaporte = st.text_input('Passaporte')
  
  def verificarNome(names):
    InterpolList = req.get('https://cspinheiro.github.io/interpol.json')
    lista_fuzzy = process.extract(names, InterpolList)
    resultado = []
    for sublista in lista_fuzzy: 
        if sublista[1] > 80:
          resultado.append(sublista)
    if len(resultado) == 0:
        return True
    return False

  res = req.get('https://felps2003.github.io/irq.json')
  paises = ['irq','fra']
  def verificacaoPassaporte(numPassaporte):
    match = re.search('(?i)(\D{3})', numPassaporte.lower())
    api = (match[0])
    if api in paises:
      x = (api)
      dict_api = res.json()[x]
      for item in dict_api:
          for value in item.values():
            if str(numPassaporte) == str(x)+str(value):
              return False
      return True

      
  def verificacao(names,numPassaporte):
    imigrante = verificarNome(names)
    passaporte = verificacaoPassaporte(numPassaporte)
    if passaporte == False or imigrante == False:
      st.error('Prenda o individuo, possivel terrorista')
    elif passaporte == True and imigrante == True:
      st.success('Imigração permitida')
    else:
      st.warning('Não existe esse pais na base de dados')
          
  if st.button("Verificar"):
    verificacao(names,numPassaporte)

if __name__=='__main__': 
    main()  

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-05-03 19:30:51.908 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.185.193.185:8501

npx: installed 22 in 4.13s
your url is: https://ripe-pianos-smile-35-185-193-185.loca.lt
2022-05-03 19:31:16.293 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/scriptrunner/script_runner.py", line 443, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 66, in <module>
    main()
  File "/content/app.py", line 63, in main
    verificacao(names,numPassaporte)
  File "/content/app.py", line 54, in verificacao
    passaporte = verificacaoPassaporte(numPassaporte)
  File "/content/app.py", line 44, in verificacaoPassaporte
    dict_api = res.json()[x]
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 898, in json
    return complexjson.loads(self.text, **kwargs)
  File "/usr/lib/pyt